In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

24/08/30 21:39:38 WARN Utils: Your hostname, JPEnvy resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/30 21:39:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 21:39:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

lines = spark.read.text("sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))

In [3]:


ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

24/08/30 21:07:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/30 21:07:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/08/30 21:07:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Root-mean-square error = 1.871181885766001


ALS_afac1d079b8d

In [5]:
# from pyspark import SparkConf
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, collect_list

# import time


# def create_spark_session() -> SparkSession:
#     conf = SparkConf().set("spark.driver.memory", "8g")

#     spark_session = SparkSession\
#         .builder\
#         .master("local[4]")\
#         .config(conf=conf)\
#         .appName("Spark UI Tutorial") \
#         .getOrCreate()

#     return spark_session


# if __name__ == '__main__':

#     spark = create_spark_session()

#     test_df = spark.createDataFrame([
#         (1, 'a'),
#         (2, 'b'),
#         (3, 'c'),
#         (4, 'd'),
#         (5, 'e'),
#         (6, 'f'),
#         (7, 'g'),
#         (8, 'h'),
#         (9, 'i'),
#         (10, 'j')
#     ], ["number", "letter"]).cache()

#     test_df.show(truncate=False)

#     test_df \
#         .withColumn("mod", col("number") % 2) \
#         .groupBy("mod") \
#         .agg(collect_list("letter").alias("letter")) \
#         .show(truncate=False)

#     # For UI to stick
#     time.sleep(1000000)